In [277]:
import pandas as pd
import numpy as np


####solver
from scipy.optimize import fsolve


import matplotlib.pyplot as plt
%matplotlib inline


# Classes

## Ho-Lee Model

In [278]:
class HoLee(object):
    
    '''
    Class to calibrate Binamial Tree to fit the interest rate
    using Ho Lee implementation
    ============================
    n: number of time period
    T: number of years. 
    dt: T/n
    zcb: array, price of zero coupon bonds. 
        Make sure these are ZCB. Only check is if zcb >1, then devide by 100. 
        
    sigma = Annualised Volatility (standard deviation)
    
    =============================
    Assumption: dt is constant
    Future development ideas: 
        If dt between each price not constant, 
        implement raw interpolation  
    
    '''
    
    def __init__(self, zcb, T,  sigma ):
        
        self.zcb = np.array(zcb) # Array
        self.n = len(self.zcb) 
        self.T = T
        self.sigma = sigma # Annualised volatility 
        self.dt = T/self.n
        
        self.rates = np.zeros((self.n, self.n))
        self.thetas = np.nan # store theta's value once calibrated
        
        # if ZCB > 1 ==> ZCB quoted per $100. 
        if self.zcb[-1] > 1.0:
            self.zcb /= 100
        
        # Extract first interest rate (Trivial)
        self.rates[0,0] = -np.log(self.zcb[0])/self.dt
        
        self.fit_theta()
        
        # Ideas, if dt not fixed, insert array along zcb to have time. 


        
    @staticmethod
    def forward_tree(r0,sigma,dt,thetas):
        """
        Forward tree valuation
        Works for both Ho Lee and BDT model (change r_0 to ln r_0)

        Args:
            r0 (float): _description_
            sigma (float): _description_
            dt (float): _description_
            thetas (array / list): _description_

        Returns:
             nxn array: interest rates
             
        =========================
        Uses backward tree for Bond evaluation
        Future area of improvement, maybe use
        backward_tree method
        """
        n = len(thetas)
        tree_rate = np.zeros((n+1,n+1))
        tree_rate[0,0] = r0
        tree_zcb = np.zeros((n+2, n+2))
        tree_zcb[:, -1] = 1.0 # Could be 100.0

        for i in range(n):
            tree_rate[0,i+1] = tree_rate[0,i] + thetas[i] * dt \
                + sigma * np.sqrt(dt)
            
            
            # Vectorise calculation by -2 sigma on each row
            # (column-wise operation)
            
            # ignore first row (already calculated)
            tree_rate[1:i+2,i+1] = tree_rate[0,i+1] \
                - 2 * np.arange(1,i+2) * sigma * np.sqrt(dt)
            
        # Calculate ZCB backward
        

        for i in np.arange(n, -1, -1):

            
            tree_zcb[0:i+1,i] = np.exp(-tree_rate[:i+1,i] * dt) \
                * 0.5 * (tree_zcb[0:i+1,i+1] + tree_zcb[1:i+2,i+1])
            

        return tree_rate, tree_zcb
            
    def fit_theta(self):
        """
        Find theta parameters
        -------------------------
        In financial mathematics, the Ho–Lee model is a short rate model widely used 
        in the pricing of bond options, swaptions and other interest rate derivatives,
        and in modeling future interest rates. It was developed in 1986 by Thomas Ho and Sang Bin Lee.
        (from Wikepedia: https://en.wikipedia.org/wiki/Ho%E2%80%93Lee_model)
        
        - Risk Neutral probability = 0.5
        
        Drawback: Ho-Lee model allows negative interest rate
        
        Great ressource: 
        https://www.bensblog.tech/fixed_income/HoLee_Model/
        """
        thetas=[]

        r0 = self.rates[0,0]
        
        
        for i in self.zcb[1:]:
            p0=i
            func = (lambda t: self.forward_tree(r0,self.sigma,self.dt,thetas+[t])[1][0,0]-p0)
            new_theta=fsolve(func,0.001)
            thetas.append(new_theta[0])
            
        self.thetas = thetas
        self.rates = self.forward_tree(r0,self.sigma,self.dt,thetas)[0]


## Simple Black Derman Toy

In mathematical finance, the Black–Derman–Toy model (BDT) is a popular short rate model
        used in the pricing of bond options, swaptions and other interest rate derivatives.
        
        Wikipedia: 
        https://en.wikipedia.org/wiki/Black%E2%80%93Derman%E2%80%93Toy_model
        

In [279]:
class BlackDermanToy(object):
    
    '''
    Class to calibrate Binamial Tree to fit the interest rate
    using  Black Derman Toy implementation
    ============================
    n: number of time period
    T: number of years. 
    dt: T/n
    zcb: array, price of zero coupon bonds. 
       
    sigma = vol of log interest rate!(standard deviation)
    
    =============================
    Assumption: dt is constant
    Future development ideas: 
        If dt between each price not constant, 
        implement raw interpolation  
    
    =============================

    '''
    
    def __init__(self, zcb, T,  sigma ):
        
        self.zcb = np.array(zcb) # Array
        self.n = len(self.zcb) 
        self.T = T
        self.sigma = sigma # vol of log interest rate!
        self.dt = T/self.n
        
        self.rates = np.zeros((self.n, self.n))
        self.thetas = np.nan # store theta's value once calibrated
        
        # if ZCB > 1 ==> ZCB quoted per $100. 
        if self.zcb[-1] > 1.0:
            self.zcb /= 100
        
        # Extract first interest rate (Trivial)
        self.rates[0,0] = -np.log(self.zcb[0])/self.dt
        
        
        self.fit_theta()
        


        
    @staticmethod
    def forward_tree(r0,sigma,dt,thetas):
        """
        Forward tree valuation
        Args:
            r0 (float): _description_
            sigma (float): _description_
            dt (float): _description_
            thetas (array / list): _description_

        Returns:
             nxn array: interest rates
             
        =========================
        Uses backward tree for Bond evaluation
        Future area of improvement, maybe use
        backward_tree method
        """
        n = len(thetas)
        tree_rate = np.zeros((n+1,n+1))
        tree_rate[0,0] = np.log(r0)
        tree_zcb = np.zeros((n+2, n+2))
        tree_zcb[:, -1] = 1.0 # Could be 100.0

        for i in range(n):
            tree_rate[0,i+1] = tree_rate[0,i] + thetas[i] * dt \
                + sigma * np.sqrt(dt)
            
            
            # Vectorise calculation by -2 sigma on each row
            # (column-wise operation)
            
            # ignore first row (already calculated)
            tree_rate[1:i+2,i+1] = tree_rate[0,i+1] \
                - 2 * np.arange(1,i+2) * sigma * np.sqrt(dt)
            
        # Calculate ZCB backward
        

        for i in np.arange(n, -1, -1):

            r = np.exp(tree_rate[:i+1,i])
            tree_zcb[0:i+1,i] = np.exp(-r * dt) \
                * 0.5 * (tree_zcb[0:i+1,i+1] + tree_zcb[1:i+2,i+1])
            
        
        
        
        # z_i = ln(r_i) <=> r_i = exp(z_i)
        # replace 1.0 by 0.0 
        tree_rate = np.exp(tree_rate)
        for i in range(len(tree_rate)):
            tree_rate[i+1:,i] = 0
       
        return tree_rate, tree_zcb
            
    def fit_theta(self):
        """
        Find theta parameters
        -------------------------
        
        ----------------------------------
        Note that differently form the Ho-Lee model, now sigma is the
        vol of log-interest rates. 
        - z_i = log(r_i) 
        - Risk Neutral probability = 0.5
        ----------------------------------
        Drawback: 
         - No analytical solution
        
        Great ressource: 
        https://www.bensblog.tech/fixed_income/HoLee_Model/
        """
        thetas=[]
        r0 = self.rates[0,0]
        
        for i in self.zcb[1:]:
            p0=i
            func = (lambda t: self.forward_tree(r0,self.sigma,self.dt,thetas+[t])[1][0,0]-p0)
            new_theta=fsolve(func,.001)
            thetas.append(new_theta[0])
            
        self.thetas = thetas
        self.rates = self.forward_tree(r0,self.sigma,self.dt,thetas)[0]


## Cap, Floor and Swaption

In [280]:
class Option_IR:
    
    def __init__(self,rate_obj, T, n):
        ''' in case I need to write something'''
        
        self.T = T          # Years
        self.n = n          # n

        self.dt = T/n
        
        
        self.rate_obj = rate_obj       
        self.zcb = rate_obj.zcb
        self.tree_rates = rate_obj.rates[:n+1, :n+1] # no need for self here
       
       
       
        # Set fair swap rate
        self.c_swap = self._swap_rate()

        
        
        
    def cash_flow(self, c=np.nan, notional=100.0, otype= "cap"):
        """
        Calculate Binomial tree for a cap
        c: swap rate (similar to Strike)
        cp str : cap/ floor flag (call/put equivalent)
        notional: (=N in the formula)
        return: tree cash flow
        """
                
        tree_ctns = self.ctns_rate(self.tree_rates, self.dt)
        
        tree_cf = np.zeros((self.n+1,self.n+1)) # create an empty array for Cash-Flow tree
        if otype =="cap":
            tree_cf = self.dt * notional * np.maximum(tree_ctns - c, 0)
        
        elif otype =="floor":
            tree_cf = self.dt * notional * np.maximum(c - tree_ctns, 0)
        
        elif otype == "swap":
            tree_cf = self.dt * notional * (tree_ctns - c)
        
            for i in range(self.n):
                tree_cf[i+1:, i] = 0

        else:
            print("No option type inputed, \n Please choose:")
            print("1.cap \n 2. floor \n 3. swap")
            return
        return tree_cf
    
    
    
    def option(self, c, notional , otype= "cap"):

        cash_flow = self.cash_flow(c, notional, otype) 
        
        rates = self.tree_rates
        
        tree = self.backward_tree(cash_flow, rates, self.dt)
        
        return tree
        
    def fair_swap(self):
        

        return self.option(self.c_swap, 100, "swap")



    def swaption(self, t):
        
        a = self.fair_swap()
                
        a = a[:t+1, :t+1]
        a[:,-1] = np.where(a[:,-1]-self.c_swap>0, a[:,-1],0)
   
        tree = np.zeros((t+1, t+1))
                #intresic value
        tree[:,-1] = a[:,-1]

        p =0.5
        for i in range(t,0,-1):
            tree[:i,i-1] = np.exp(-self.tree_rates[:i, i-1] * self.dt) \
                *((p * tree[:i,i] + (1-p) * tree[1:i+1,i]))
        
       
        return tree

    def _swap_rate(self):
        """
        Find Fair swap rate analytically
        pro: Analitical solution

        Args:
            zcb (list / array): zero coupon bond

        Returns:
            float: fair swap rate
            
        ===========================================
        NB: I could create a bigger object so I don't have
        to manually input the zcb array. Alternatively, 
        I could calculate the ZCB from the array.  
        """
        
        k = len(self.tree_rates)

        self.c_swap = 1/self.dt * (1-self.zcb[k-1])/ (sum(self.zcb[:k]))
       
        return self.c_swap

    def fit_swap_rate(self):
        """
        Depreciated: 
        _swap_rate() is now used at instantiation
        Could still be useful on some occasions, 
        but will be ignored for now
        ---------------------------------------------
        Find fair swap rate using a numerical method. 
        Pro: no zcb needed
        Con: Not an analytical solution

        Returns:
            float: fair swap rate
        """
        
        # Notional amount is irrelevant
        func = (lambda t: self.option(t, 1.0, "swap")[0,0])
        c = fsolve(func, 0.001)
        self.swap_rate = c[0]
        return c[0]

    @staticmethod
    def backward_tree(tree_cf, tree_rates, dt):
        """
        Calculate whole tree backward
        ==============================
        Args:
            tree_cf (nxn np.array): Cash Flow
            tree_rates (nxn np.array): rates from Ho Lee or Black Derman Toy
            dt (float): time step (assume constant)

        Returns:
            _type_: _description_
        """
        n = len(tree_rates)
        p = 0.5 # probability
        
        tree_eu = np.zeros((n, n))
                
        #intresic value
        tree_eu[:,-1] = np.exp(-tree_rates[:,-1] * dt) * tree_cf[:,-1]
        

        for i in range(n-1,0,-1):
            tree_eu[:i,i-1] = np.exp(-tree_rates[:i, i-1] * dt) \
                *((p * tree_eu[:i,i] + (1-p) * tree_eu[1:i+1,i]) \
                    +tree_cf[:i, i-1])

        return tree_eu
    
    @staticmethod
    def ctns_rate(rate, time):
    
        return (np.exp(rate * time)-1) / time
    
    

    


### Future ideas:

Input a numerical solver method

In [282]:
def secant_method(f, estimate = 0.05):
    """Return the root calculated using the secant method."""
    
    iterations=5e2
    
    p0 = estimate * 1.0
    funcall = 0
    eps =1e-4
    p1 = estimate*(1+eps)
    p1 += (eps if p1 >= 0 else -eps)
    
    funcalls = 0
    
    q0 = f(p0)
    funcalls += 1
    q1 = f(p1)
    funcalls += 1

    if abs(q1) < abs(q0):
        p0, p1, q0, q1 = p1, p0, q1, q0
    for itr in range(int(iterations)):
        if q1 == q0:
            if p1 != p0:
                msg = "Tolerance of %s reached." % (p1 - p0)
                if disp:
                    msg += (
                        " Failed to converge after %d iterations, value is %s."
                        % (itr + 1, p1))
                    raise RuntimeError(msg)
                warnings.warn(msg, RuntimeWarning)
            p = (p1 + p0) / 2.0
    else:
            if abs(q1) > abs(q0):
                p = (-q0 / q1 * p1 + p0) / (1 - q0 / q1)
            else:
                p = (-q1 / q0 * p0 + p1) / (1 - q1 / q0)

    return p
    

# Testing

## Data
I will be using data from "Fixed Income Securities" by Pietro Veronesi
The examples are taken from chapter 11.

In [77]:
# Table 10.11 Zero Coupon Bond Prices on January 8, 2002,
# Source The Wall Street Journal
# Taken from Fixed Income Securities - Pietro Veronesi || p.370


maturity = np.arange(0.5,6.0,0.5)

price = [99.1338, 97.8925, 96.1462,
         94.1011, 91.7136, 89.2258,
         86.8142, 84.5016, 82.1848,
         79.7718, 77.4339]

yield_ = [1.74, 2.13, 2.62, 3.04,
          3.46,3.8,4.04,4.21,4.36, 
          4.52,4.65]
sigma_hl = 0.0173
sigma_bdt = 0.2142
df = pd.DataFrame([maturity, price, yield_]).T
df.columns = ["Maturity", "Price", "Yield"]

In [79]:
zcb_02 = [i/100 for i in price]

#### Table: 11.2; p 384

In [80]:
# Theta from book
theta_hl = [0.015674,0.021824,0.014374,0.017324,0.007873,0.000423,-0.000628,0.004322,0.009271,0.001202]
theta_bdt = [0.7182,  0.6916, 0.3348, 0.3379, 0.1182, -0.023, -0.0438, 0.0455, 0.1281, -0.0126]


In [81]:
# Fit Ho - Lee
hl=HoLee(price, 5.5, 0.0173)

# Fit Black Derman Toy
bdt=BlackDermanToy(price, 5.5, 0.2142)


In [82]:
# Ho-Lee
temp = pd.DataFrame([theta_hl, hl.thetas]).T
temp["diff"]= temp.iloc[:,0] - temp.iloc[:,1]
temp.columns = ["Book", "Value", "Diff"]
temp

,Book,Value,Diff
0,0.015674,0.015678,-3.949304e-06
1,0.021824,0.021822,1.646897e-06
2,0.014374,0.014375,-9.834329e-07
3,0.017324,0.017319,4.944986e-06
4,0.007873,0.007879,-5.767581e-06
5,0.000423,0.000421,1.904846e-06
6,-0.000628,-0.000629,1.018752e-06
7,0.004322,0.004323,-1.008647e-06
8,0.009271,0.009272,-6.841671e-07
9,0.001202,0.001200,1.663697e-06


In [83]:
# Black Derman Toy
temp = pd.DataFrame([theta_bdt, bdt.thetas]).T
temp["diff"]= temp.iloc[:,0] - temp.iloc[:,1]
temp.columns = ["Book", "Value", "Diff"]
temp

,Book,Value,Diff
0,0.7182,0.718322,-1.217830e-04
1,0.6916,0.691521,7.901000e-05
2,0.3348,0.334844,-4.433694e-05
3,0.3379,0.337824,7.616040e-05
4,0.1182,0.118293,-9.296763e-05
5,-0.0230,-0.023000,-4.556782e-07
6,-0.0438,-0.043815,1.502993e-05
7,0.0455,0.045526,-2.575932e-05
8,0.1281,0.128076,2.381271e-05
9,-0.0126,-0.012640,3.971751e-05


### Observations:
Thetas are similar!

## Caps and Floor

Caps are like european calls <br>
Floors are like european puts

In [190]:
option_ir = Option_IR(bdt, 1,2)

##### Cash Flow tree: (p.390)

In [191]:
# Cash flow( Please note it arrives a T+1)
pd.DataFrame(option_ir.cash_flow(0.025,100,'cap'))

,0,1,2
0,0.0,0.210221,1.162091
1,0.0,0.000000,0.526143
2,0.0,0.000000,0.058934


##### Cap Value Tree: (p.391)

In [192]:
# Cap
pd.DataFrame(option_ir.option(0.025,100,'cap'))

,0,1,2
0,0.647173,1.021151,1.134721
1,0.000000,0.284504,0.516961
2,0.000000,0.000000,0.058173


##### Cash Flow, 5-year Cap: Panel A, p 392

In [193]:
option_bdt = Option_IR(bdt, 4.5,9)

In [194]:
pd.DataFrame(option_bdt.cash_flow(0.025,100,'cap'))

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.210221,1.162091,2.083010,3.370399,4.484088,5.373482,6.323552,7.828665,10.132145
1,0.0,0.000000,0.526143,1.201369,2.142664,2.954517,3.601259,4.290568,5.379334,7.038055
2,0.0,0.000000,0.058934,0.554974,1.245054,1.838918,2.311141,2.813607,3.605510,4.807917
3,0.0,0.000000,0.000000,0.080132,0.587037,1.022552,1.368393,1.735930,2.314242,3.190160
4,0.0,0.000000,0.000000,0.000000,0.103704,0.423694,0.677543,0.947072,1.370663,2.011063
5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.170231,0.368261,0.679208,1.148673
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.171455,0.516293
7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.051692
8,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


##### Cash Flow, 5-year Cap: Panel B, p 392

In [195]:
pd.DataFrame(option_bdt.option(0.025,100,'cap'))

,0,1,2,3,4,5,6,7,8,9
0,9.439293,12.188350,15.098409,17.349262,18.926982,19.438340,18.890260,17.286536,14.326108,9.096741
1,0.000000,6.855192,9.213803,11.251749,12.762027,13.423830,13.247467,12.249406,10.218269,6.499382
2,0.000000,0.000000,4.644180,6.450871,7.890378,8.680841,8.819973,8.328348,7.056781,4.533294
3,0.000000,0.000000,0.000000,2.841199,4.134290,5.003546,5.400162,5.317499,4.649563,3.054534
4,0.000000,0.000000,0.000000,0.000000,1.463428,2.242857,2.789242,3.028833,2.831127,1.947552
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.516212,0.924161,1.302093,1.465485,1.121766
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.120999,0.232017,0.444320,0.507332
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012515,0.025249,0.051028
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Cash Flow - 5Y Swap Tree: Panel A: p 394

In [196]:
pd.DataFrame(option_bdt.cash_flow(0.0449,100, "swap"))

,0,1,2,3,4,5,6,7,8,9
0,-1.371231,-0.784779,0.167091,1.088010,2.375399,3.489088,4.378482,5.328552,6.833665,9.137145
1,0.000000,-1.168447,-0.468857,0.206369,1.147664,1.959517,2.606259,3.295568,4.384334,6.043055
2,0.000000,0.000000,-0.936066,-0.440026,0.250054,0.843918,1.316141,1.818607,2.610510,3.812917
3,0.000000,0.000000,0.000000,-0.914868,-0.407963,0.027552,0.373393,0.740930,1.319242,2.195160
4,0.000000,0.000000,0.000000,0.000000,-0.891296,-0.571306,-0.317457,-0.047928,0.375663,1.016063
5,0.000000,0.000000,0.000000,0.000000,0.000000,-1.011403,-0.824769,-0.626739,-0.315792,0.153673
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.197875,-1.052174,-0.823545,-0.478707
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.365280,-1.196974,-0.943308
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.471926,-1.285125
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.536870


5Y - Swap Value Tree: Panel B: p 394

In [230]:
pd.DataFrame(option_bdt.fair_swap())

,0,1,2,3,4,5,6,7,8,9
0,-2.597431e-14,4.266413,8.183716,11.381237,13.862339,15.224889,15.500114,14.715964,12.585224,8.204431
1,0.000000e+00,-1.526202,2.041017,5.044557,7.480540,9.027730,9.715254,9.578343,8.416391,5.581576
2,0.000000e+00,0.000000,-2.791639,0.045441,2.440907,4.143350,5.178066,5.580019,5.208297,3.596188
3,0.000000e+00,0.000000,0.000000,-3.831919,-1.470584,0.357909,1.674516,2.510375,2.765778,2.102913
4,0.000000e+00,0.000000,0.000000,0.000000,-4.467708,-2.539433,-0.999787,0.177311,0.920777,0.985065
5,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-4.736601,-3.023723,-1.582764,-0.464760,0.151173
6,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,-4.545787,-2.903283,-1.500770,-0.469292
7,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.889993,-2.272961,-0.930076
8,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.847159,-1.271792
9,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.524946


Table 11.9 - A 2-Year Payer Swaption

Example 11.6

In [276]:
option_bdt.swaption(4)

13.862338581086318


array([[ 3.40669969,  5.11220111,  7.40599046, 10.32723181, 13.86233858],
       [ 0.        ,  1.76073161,  2.96771058,  4.84202763,  7.48054035],
       [ 0.        ,  0.        ,  0.59166307,  1.19881511,  2.44090682],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

# Future ideas: 

- Code has been vectorised to avoid double loop. Nonetheless some part of the code could still be improved

- Try different solver

- Implement full Black Derman Toy model (not just the simple one)